<!--
tags: [tutorial]
description: Conversation patterns
-->

# Conversation Patterns

In the [last chapter](./code-executors) we used two-agent conversation, which
can be started by the `initiate_chat` method.
Two-agent chat is a useful conversation pattern but AutoGen offers more.
In this chapter, we will first dig a little bit more into the two-agent 
chat pattern, then we will show you several conversation patterns that involve 
more than two agents.


In [1]:
import os
from autogen import initiate_chats, ConversableAgent, GroupChat, GroupChatManager

Neither powershell nor pwsh is installed.


## Two-Agent Chat

Two-agent chat is the simplest from of conversation pattern. We have already
seen multiple examples of two-agent chat in the previous chapters
but there are still some details that we haven't covered yet.
The following figure illustrates how two-agent chat works.

```{=mdx}
![Two-agent chat](./assets/two-agent-chat.png)
```

Besides the conversation itself, there is also a summarizer that can be used
to produce a summary of the conversation. The summarizer is optional and by
default the summary is the last message of the conversation.
This can be changed by setting the `summarizer` parameter of the `initiate_chat`.

In [5]:
agent_a = ConversableAgent(
    name="Agent A",
    system_message="You are a student willing to learn.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)
agent_b = ConversableAgent(
    name="Agent B",
    system_message="You are a math teacher.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

agent_a.initiate_chat(
    agent_b, message="What is triangle inequality?", max_turns=2, summary_method="reflection_with_llm"
)

Agent A (to Agent B):

What is triangle inequality?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent B (to Agent A):

Triangle inequality theorem is a fundamental principle in geometry that states that the sum of the lengths of any two sides of a triangle must always be greater than the length of the third side. In a triangle with sides of lengths a, b, and c, the theorem can be written as:

a + b > c
a + c > b
b + c > a

Each of these represents the condition for one specific side (a, b, or c). All must be true for a triangle to exist.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


Agent A (to Agent B):

Thank you for the explanation. This theorem helps in understanding the basic properties of a triangle. It can also be useful when solving geometric problems or proving other mathematical theorems. Can you give me an example of how we can use the triangle inequality theorem?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent B (to Agent A):

Absolutely! Here's an example:

Suppose you're given three line segments with lengths 10, 7, and 3 units. The question is: "Can these three line segments form a triangle?"

To answer this, you would use the triangle inequality theorem. Adding any two side lengths together should be greater than the third:

- For sides 10 and 7: 10 + 7 = 17, which is larger than 3.
- For sides 10 and 3: 10 + 3 = 13, which is larger than 7.
- For sides 7 and 3: 7 + 3 = 10, which is equal to the length of the third side (10), but not greater.

So, these three lines cannot form a t

ChatResult(chat_history=[{'content': 'What is triangle inequality?', 'role': 'assistant'}, {'content': 'Triangle inequality theorem is a fundamental principle in geometry that states that the sum of the lengths of any two sides of a triangle must always be greater than the length of the third side. In a triangle with sides of lengths a, b, and c, the theorem can be written as:\n\na + b > c\na + c > b\nb + c > a\n\nEach of these represents the condition for one specific side (a, b, or c). All must be true for a triangle to exist.', 'role': 'user'}, {'content': 'Thank you for the explanation. This theorem helps in understanding the basic properties of a triangle. It can also be useful when solving geometric problems or proving other mathematical theorems. Can you give me an example of how we can use the triangle inequality theorem?', 'role': 'assistant'}, {'content': 'Absolutely! Here\'s an example:\n\nSuppose you\'re given three line segments with lengths 10, 7, and 3 units. The questio

## Sequential Two-Agent Chats

The name of this pattern is self-explanatory -- it is a sequence of
two-agent chats.
This pattern is useful when you can break down a complex task into 
interdependent sub-tasks. 
The figure below illustrate how this pattern works.

```{=mdx}
![initiate_chats](./assets/initiate_chats.png)
```

In this pattern, the a pair of agents first start a conversation,


In [2]:
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

results = initiate_chats(
    [
        {
            "sender": number_agent,
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
        },
        {
            "sender": number_agent,
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
        {
            "sender": number_agent,
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
        {
            "sender": number_agent,
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
14

With the following carryover: 


********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
These are my numbers

With the following carryover: 
16

********************************************************************************
Number_Agent (to Mul

## Group Chat

```{=mdx}
![group_chat](./assets/group-chat.png)
```

In [3]:
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent],
    messages=[],
    max_round=16,
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 40, I want to turn it into 12.",
)

Number_Agent (to chat_manager):

My number is 40, I want to turn it into 12.

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

20

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

19

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

9.5

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

10.5

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

21

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

20

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

10

-------------------------------------------------------------------------------

## Nested Chat

[To be completed...]